**<h2>Import Librarys<h2>**

In [1]:
#GEE Libraries - Installing using Conda
import ee #GEE library
import geemap #GEE library to combine with spatial data
import folium #Folium Library to create a Leaflet visualization map
import geopandas as gpd
import pandas as pd
from datetime import datetime

#Path Libraries
import pathlib as Path #import os # pathlib is better than os, avoid the use of this library

ee.Initialize() #GEE initialization

**<h2>Change Path Area and Ground data Locations<h2>**

In [2]:
current_path=Path.Path.cwd().parent
campinas=current_path.joinpath('data','campinas', 'gis_data','shapefiles','plots','PLOTS4326.shp') #PLOTS4326.shp"

**<h2>Define Input Parameters (Dates and attribute to filter areas)<h2>**

In [3]:
#Name of the total area
area = campinas

#Name of the attribute field to filter
selection = 'names'

#Dates to filter the imagery
startDate = '2021-09-12'
endDate = '2021-09-14'

d1 = datetime.strptime(startDate, "%Y-%m-%d")
d2 = datetime.strptime(endDate, "%Y-%m-%d")

AOI_total = geemap.shp_to_ee(area)

abs((d2 - d1).days)



2

**<h2>Functions to Compute Multiespectral indexes<h2>**

In [4]:
#Adding a NDVI band
def indexes(image):
  #Define Satellite bands names
  BLUE = image.select('B2')
  GREEN = image.select('B3')
  RED = image.select('B4')
  REDG = image.select('B5')
  NIR = image.select('B8')

  #Multiespectral indexes expressions
  NDVI = NIR.subtract(RED).divide(NIR.add(RED)).rename('NDVI')
  NDRE = NIR.subtract(REDG).divide(NIR.add(REDG)).rename('NDRE')
  GNDVI = NIR.subtract(GREEN).divide(NIR.add(GREEN)).rename('GNDVI')
  BNDVI = NIR.subtract(BLUE).divide(NIR.add(BLUE)).rename('BNDVI')
  NPCI = RED.subtract(BLUE).divide(RED.add(BLUE)).rename('NPCI')
  GRVI = GREEN.subtract(RED).divide(GREEN.add(RED)).rename('GRVI')
  NGBDI = NIR.subtract(GREEN.add(RED)).divide(NIR.add(GREEN).add(RED)).rename('NGBDI')

  return image.addBands([NDVI,NDRE,GNDVI,BNDVI,NPCI,GRVI,NGBDI])

**<h2>Adding the image collection with cloud mask<h2>**

In [5]:
#Image collection Sentinel-2
imagecollection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate(startDate, endDate) \
    .map(indexes) \
    .filterBounds(AOI_total)


image = ee.Image(imagecollection.first())

#Cloud mask
cloudProb = image.select('MSK_CLDPRB')
snowProb = image.select('MSK_SNWPRB')
cloud = cloudProb.lt(5)
snow = snowProb.lt(5)
scl = image.select('SCL') 
shadow = scl.eq(3); # 3 = cloud shadow
cirrus = scl.eq(10); # 10 = cirrus
#Cloud probability less than 5% or cloud shadow classification
mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))

image = image.updateMask(mask)
date = ee.Date(image.get('system:time_start')).format("yyyy-MM-dd").getInfo() #Image date

In [6]:
date

'2021-09-13'

**<h2>Getting the multiespectral indexes median values per plot area in a final Remote Sensing Dataset<h2>**

In [7]:
shape = gpd.read_file(area)
attable = pd.DataFrame(shape,columns=[selection])
columns = ["Area", "ID", "Date", "NDVI","NDRE","GNDVI","BNDVI","NPCI","GRVI","NGBDI"]

i = 0
rstable= pd.DataFrame()

for i in range(len(attable.index)):
    namarea = attable.iloc[i,0]
    AOI = AOI_total.filter(ee.Filter.eq(selection, namarea))
    
    stats = image.reduceRegion(
      reducer = ee.Reducer.median(),
      geometry= AOI.geometry(),
      scale = 1,
      maxPixels = 1e10
      )

    data = [columns, [namarea, shape.iloc[i,0], date, stats.get('NDVI').getInfo(), stats.get('NDRE').getInfo(), stats.get('GNDVI').getInfo(), stats.get('BNDVI').getInfo(), stats.get('NPCI').getInfo(), stats.get('GRVI').getInfo(),  stats.get('NGBDI').getInfo()] ]

    column_names = data.pop(0)
    df = pd.DataFrame(data, columns=column_names)
    rstable = rstable.append(df) 


rsexport=current_path.joinpath('data','campinas','remote_sensing',date +'.xlsx') #File to export
rstable.to_excel(rsexport)  
rstable


,Area,ID,Date,NDVI,NDRE,GNDVI,BNDVI,NPCI,GRVI,NGBDI
0,G01,1,2021-09-13,0.652695,0.455176,0.583042,0.698042,0.082309,0.110552,0.358072
0,G02,2,2021-09-13,0.636407,0.431107,0.573946,0.682094,0.083598,0.100436,0.342782
0,G03,3,2021-09-13,0.532353,0.364837,0.483587,0.568742,0.042627,0.057260,0.201429
0,G04,4,2021-09-13,0.533456,0.379597,0.487460,0.545582,0.015474,0.060927,0.209825
0,G05,5,2021-09-13,0.486881,0.318538,0.483677,0.576665,0.114159,0.016917,0.179068
0,G06,6,2021-09-13,0.505437,0.336176,0.474824,0.556149,0.071347,0.040163,0.189900
0,G07,7,2021-09-13,0.763414,0.619671,0.700233,0.797611,0.093352,0.128523,0.529163
0,G10,10,2021-09-13,0.698852,0.481842,0.607294,0.718297,0.049446,0.149746,0.407166
0,G09,9,2021-09-13,0.699418,0.501363,0.598304,0.705492,0.001702,0.180014,0.393517
0,G08,8,2021-09-13,0.733344,0.535541,0.620944,0.689546,-0.052814,0.196443,0.444293


**<h2>Map visualization<h2>**

In [8]:
Map = geemap.Map()
Map.centerObject(AOI_total, 30)

visParams = {'color': 'blue'}
Map.addLayer(AOI_total, visParams,'Study Area')

palette = ['red', 'yellow','green']

ndvi = image.select('NDVI').clip(AOI_total).updateMask(mask)
ndviVis = {'min':-1, 'max':1, 'palette': palette }
Map.addLayer(ndvi, ndviVis, 'NDVI Composite')

ndre = image.select('NDRE').clip(AOI_total).updateMask(mask)
Map.addLayer(ndre, ndviVis, 'NDRE Composite')

Map

Map

Map(center=[3.0689142976407218, -76.47393986146815], controls=(WidgetControl(options=['position', 'transparent…